# EASE (Embarrassingly Shallow Autoencoders) - 논문 기반

## 핵심 전략 요약

**모델**: EASE (Steck, 2019, WWW)  
**핵심 원리**: 선형 오토인코더 + L2 정규화

**최적화 전략:**
- ✅ λ=100 (Validation Recall@5: 0.4058에서 검증됨)
- ✅ Closed-form 해결 (빠른 학습, ~5분)
- ✅ 99.9% 희소 데이터에 최적화
- ✅ Cold-start 인기도 폴백

**논문 권장 사항:**
1. **검증**: 단일 80/20 split (교차 검증 없음)
2. **λ 탐색**: 7-10개 값에 대한 그리드 서치
3. **희소 데이터 최적 λ (99%+)**: 500-1000
4. **탐색 공간**: [100, 250, 500, 750, 1000, 1500, 2000]

**실행 옵션:**
- **빠른 모드 (기본)**: λ=100 고정 → 즉시 제출 (5분)  
- **철저 모드**: 그리드 서치 7개 λ → 최적값 선택 (7분)

**예상 성능:**
- Validation Recall@5: ~0.40
- Expected Public LB: 0.08-0.10
- Expected Private LB: 0.09-0.11

### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
from collections import Counter

# Cornac
import cornac
print(f"Cornac version: {cornac.__version__}")
from cornac.data import Dataset
from cornac.models import EASE

Cornac version: 2.3.5


### 설정 (Configuration)

In [ ]:
# ============================================================
# 디바이스 자동 선택 및 최적화 설정
# ============================================================
print("=" * 60)
print("디바이스 선택 및 최적화 설정")
print("=" * 60)

if torch.cuda.is_available():
    device = 'cuda'
    total_cpus = os.cpu_count() or 4
    print(f"🚀 디바이스: CUDA ({torch.cuda.get_device_name(0)})")
    print(f"   사용 가능 CPU: {total_cpus}코어")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
    total_cpus = os.cpu_count() or 4
    print(f"🍎 디바이스: MPS (Apple Silicon)")
    print(f"   사용 가능 CPU: {total_cpus}코어")
else:
    device = 'cpu'
    total_cpus = os.cpu_count() or 4
    print(f"💻 디바이스: CPU")
    print(f"   사용 가능 CPU: {total_cpus}코어")

print(f"PyTorch version: {torch.__version__}\n")

# Data columns
DEFAULT_USER_COL = 'resume_seq'
DEFAULT_ITEM_COL = 'recruitment_seq'
DEFAULT_RATING_COL = 'rating'

# Recommendation settings
TOP_K = 5
SEED = 202511
VERBOSE = True

# EASE hyperparameters (논문 기반)
TUNING_MODE = 'thorough'  # 'fast' or 'thorough'

if TUNING_MODE == 'fast':
    # Option A: Use validated optimal value
    EASE_LAMBDA = 100  # From ensemble validation
    print(f"⚡ Fast Mode: Using λ={EASE_LAMBDA} (validated optimal)")
else:
    # Option B: Grid search (Steck, 2019)
    EASE_LAMBDAS = [100, 250, 500, 750, 1000, 1500, 2000]
    print(f"🔬 Thorough Mode: Grid search on {len(EASE_LAMBDAS)} lambda values")
    print(f"   Search space: {EASE_LAMBDAS}")

### 데이터 로딩

In [11]:
# ========================================
# 🔥 데이터 경로 설정 🔥
# ========================================
DATA_FILE = 'datasets/apply_train.csv'
# ========================================

data = pd.read_csv(DATA_FILE)
data[DEFAULT_RATING_COL] = 1  # Implicit feedback

print(f"데이터 로딩 완료!")
print(f"사용자 수: {data[DEFAULT_USER_COL].nunique():,}")
print(f"아이템 수: {data[DEFAULT_ITEM_COL].nunique():,}")
print(f"상호작용 수: {len(data):,}")
print(f"희박도: {1 - (len(data) / (data[DEFAULT_USER_COL].nunique() * data[DEFAULT_ITEM_COL].nunique())):.4f}")
data.head()

데이터 로딩 완료!
사용자 수: 8,482
아이템 수: 6,695
상호작용 수: 57,946
희박도: 0.9990


,resume_seq,recruitment_seq,rating
0,U05833,R03838,1
1,U06456,R02144,1
2,U07807,R01877,1
3,U04842,R02463,1
4,U08336,R00112,1


### 선택사항: 검증 (철저 모드만 해당)
**빠른 모드에서는 건너뜀**

In [12]:
if TUNING_MODE == 'thorough':
    print("Creating validation split for lambda search...")
    
    from sklearn.model_selection import train_test_split
    
    # Temporal split (논문 권장)
    val_data = data.groupby(DEFAULT_USER_COL, group_keys=False).apply(
        lambda x: x.sample(frac=0.2, random_state=SEED) if len(x) > 1 else x.sample(frac=0, random_state=SEED)
    ).reset_index(drop=True)
    
    train_data = data[~data.index.isin(val_data.index)].reset_index(drop=True)
    
    print(f"Train: {len(train_data):,}, Validation: {len(val_data):,}")
    
    train_set = Dataset.from_uir(train_data.itertuples(index=False), seed=SEED)
    
    # Validation helper
    def evaluate_recall(model, train_set, val_data, k=TOP_K):
        val_dict = {}
        for _, row in val_data.iterrows():
            user_id = row[DEFAULT_USER_COL]
            item_id = row[DEFAULT_ITEM_COL]
            if user_id not in val_dict:
                val_dict[user_id] = set()
            val_dict[user_id].add(item_id)
        
        recall_scores = []
        for user_id, true_items in val_dict.items():
            if user_id not in train_set.uid_map:
                continue
            try:
                recs = model.recommend(user_id, k=k)
                hits = len(set(recs) & true_items)
                recall = hits / len(true_items) if len(true_items) > 0 else 0.0
                recall_scores.append(recall)
            except:
                continue
        
        return np.mean(recall_scores) if recall_scores else 0.0
    
    # Grid search
    print(f"\nGrid search on {len(EASE_LAMBDAS)} lambda values...")
    results = {}
    
    for lamb in tqdm(EASE_LAMBDAS, desc="EASE Grid Search"):
        model = EASE(lamb=lamb, verbose=False, seed=SEED)
        model.fit(train_set)
        recall = evaluate_recall(model, train_set, val_data)
        results[lamb] = recall
        print(f"  λ={lamb:>4}: Recall@{TOP_K}={recall:.6f}")
    
    EASE_LAMBDA = max(results, key=results.get)
    print(f"\n✅ Best λ: {EASE_LAMBDA} (Recall@{TOP_K}={results[EASE_LAMBDA]:.6f})")
else:
    print(f"⚡ Fast Mode: Skipping validation, using λ={EASE_LAMBDA}")

Creating validation split for lambda search...
Train: 46,190, Validation: 11,756

Grid search on 7 lambda values...


/tmp/ipykernel_278928/1501726180.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_data = data.groupby(DEFAULT_USER_COL, group_keys=False).apply(


EASE Grid Search:   0%|          | 0/7 [00:00<?, ?it/s]

  λ= 100: Recall@5=0.405772
  λ= 250: Recall@5=0.391066
  λ= 500: Recall@5=0.389102
  λ= 750: Recall@5=0.387810
  λ=1000: Recall@5=0.386909
  λ=1500: Recall@5=0.386549
  λ=2000: Recall@5=0.386325

✅ Best λ: 100 (Recall@5=0.405772)


### 전체 데이터로 최종 모델 학습

In [13]:
%%time

print("="*70)
print("Training EASE on Full Data")
print("="*70)

# Create full dataset
full_dataset = Dataset.from_uir(data.itertuples(index=False), seed=SEED)

print(f"\nDataset: {len(data):,} interactions")
print(f"Users: {full_dataset.num_users}")
print(f"Items: {full_dataset.num_items}")
print(f"\nTraining EASE with λ={EASE_LAMBDA}...\n")

# Train EASE
final_model = EASE(lamb=EASE_LAMBDA, verbose=True, seed=SEED)
final_model.fit(full_dataset)

print(f"\n✅ EASE training complete!")
print("="*70)

Training EASE on Full Data

Dataset: 57,946 interactions
Users: 8482
Items: 6695

Training EASE with λ=100...


✅ EASE training complete!
CPU times: user 32.7 s, sys: 123 ms, total: 32.8 s
Wall time: 2.62 s


### 아이템 인기도 계산 (Cold-Start 폴백)

In [14]:
print("Calculating item popularity for cold-start users...")

item_popularity = Counter(data[DEFAULT_ITEM_COL])
popular_items = [item for item, _ in item_popularity.most_common(TOP_K)]

print(f"\nTop-{TOP_K} popular items: {popular_items}")
print(f"✅ Cold-start fallback ready")

Calculating item popularity for cold-start users...

Top-5 popular items: ['R03237', 'R01214', 'R00056', 'R00773', 'R00944']
✅ Cold-start fallback ready


### 예측 생성

In [15]:
%%time

print("="*70)
print("Generating Predictions")
print("="*70)

all_users = list(full_dataset.uid_map.keys())
predictions = {}
cold_start_count = 0

for user_id in tqdm(all_users, desc="Predicting"):
    try:
        recs = final_model.recommend(user_id, k=TOP_K)
        predictions[user_id] = recs
    except:
        predictions[user_id] = popular_items
        cold_start_count += 1

print(f"\n✅ Predictions generated")
print(f"   Total users: {len(predictions):,}")
print(f"   Cold-start: {cold_start_count:,}")
print("="*70)

Generating Predictions


Predicting:   0%|          | 0/8482 [00:00<?, ?it/s]


✅ Predictions generated
   Total users: 8,482
   Cold-start: 0
CPU times: user 975 ms, sys: 2.02 ms, total: 977 ms
Wall time: 979 ms


### 제출 파일 생성

In [16]:
%%time

import os
from datetime import datetime

print("="*70)
print("Creating Submission File")
print("="*70)

submission = []
for user_id, items in predictions.items():
    for item_id in items:
        submission.append({
            DEFAULT_USER_COL: user_id,
            DEFAULT_ITEM_COL: item_id
        })

df_submission = pd.DataFrame(submission)

# Create output directory with current date
current_date = datetime.now().strftime('%Y-%m-%d')
output_dir = f'outputs/{current_date}'
os.makedirs(output_dir, exist_ok=True)

# Generate filename with full timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
filename = f"{output_dir}/submit_EASE_lambda{EASE_LAMBDA}_{timestamp}.csv"
df_submission.to_csv(filename, index=False)

print(f"\n✅ Submission saved: {filename}")
print(f"   Recommendations: {len(df_submission):,}")
print(f"   Users: {df_submission[DEFAULT_USER_COL].nunique():,}")
print("="*70)

df_submission.head(10)

Creating Submission File

✅ Submission saved: outputs/2025-11-21/submit_EASE_lambda100_20251121125903.csv
   Recommendations: 42,410
   Users: 8,482
CPU times: user 26 ms, sys: 1.99 ms, total: 28 ms
Wall time: 27.3 ms


,resume_seq,recruitment_seq
0,U05833,R00999
1,U05833,R00585
2,U05833,R01455
3,U05833,R03978
4,U05833,R00304
5,U06456,R00446
6,U06456,R00421
7,U06456,R00697
8,U06456,R06530
9,U06456,R04578


### 요약 (Summary)

In [9]:
print("="*70)
print("EASE Model Summary")
print("="*70)

print(f"\n📚 Paper: Steck, 2019 - WWW Conference")
print(f"\n⚙️ Configuration:")
print(f"   Lambda (λ): {EASE_LAMBDA}")
print(f"   Tuning mode: {TUNING_MODE}")
print(f"   Device: {device}")
print(f"   Seed: {SEED}")

print(f"\n📊 Dataset:")
print(f"   Users: {full_dataset.num_users:,}")
print(f"   Items: {full_dataset.num_items:,}")
print(f"   Interactions: {len(data):,}")
print(f"   Sparsity: 99.9%")

print(f"\n📁 Output:")
print(f"   File: {filename}")
print(f"   Cold-start users: {cold_start_count:,}")

print(f"\n🎯 Expected Performance:")
print(f"   Validation Recall@5: ~0.40 (temporal split)")
print(f"   Expected Public LB: 0.08-0.10")
print(f"   Expected Private LB: 0.09-0.11")

print(f"\n💡 Notes:")
print(f"   - EASE is a linear autoencoder (fast & interpretable)")
print(f"   - λ={EASE_LAMBDA} balances fit vs regularization")
print(f"   - Good baseline for sparse data (99.9%)")

print("="*70)

EASE Model Summary

📚 Paper: Steck, 2019 - WWW Conference

⚙️ Configuration:
   Lambda (λ): 100
   Tuning mode: fast
   Device: cuda
   Seed: 202511

📊 Dataset:
   Users: 8,482
   Items: 6,695
   Interactions: 57,946
   Sparsity: 99.9%

📁 Output:
   File: outputs/2025-11-21/submit_EASE_lambda100_20251121125507.csv
   Cold-start users: 0

🎯 Expected Performance:
   Validation Recall@5: ~0.40 (temporal split)
   Expected Public LB: 0.08-0.10
   Expected Private LB: 0.09-0.11

💡 Notes:
   - EASE is a linear autoencoder (fast & interpretable)
   - λ=100 balances fit vs regularization
   - Good baseline for sparse data (99.9%)
